In [0]:
from google.colab import drive
drive.mount('/content/drive/')


In [0]:
!pwd
%cd "/content/drive/My Drive/c_p/"


In [0]:
import random as rn
import numpy as np
np.random.seed(42)
rn.seed(12345)
!nvidia-smi
# %load_ext tensorboard
import datetime

import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
import cv2
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Lambda, multiply, ReLU, LeakyReLU, MaxPooling2D,MaxPool2D, GaussianNoise, GlobalMaxPooling2D, GlobalAveragePooling2D,  SpatialDropout2D,  Flatten, BatchNormalization, concatenate, Input, Activation, Average, AveragePooling2D
from sklearn import svm, metrics
from sklearn.neural_network import MLPClassifier

import argparse
from sklearn import tree
import keras
from sklearn.ensemble import AdaBoostClassifier
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from collections import Counter
from tqdm.notebook import tqdm
from tensorflow.keras.initializers import Constant
from keras.utils import to_categorical
import datetime
from sklearn.metrics import plot_confusion_matrix
import pandas as pd
print("Imported")
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()



In [0]:
# from helper import Helper
# helper = Helper(mode='all')

In [0]:
# for i in range(0, 100):
#     if helper.Y_train[i] == 4:
#         plt.figure(dpi=200)
#         plt.imshow(cv2.cvtColor(helper.X_train_images[i], cv2.COLOR_BGR2RGB))
#         plt.title(helper.Y_train[i])

In [0]:
def read_data_set(path, img_path, means=None, stds=None):

    
    df = pd.read_csv(path, index_col=None)
    file_names = df['file_name']
    Y = df['annotation']
    df = df.drop(columns=['file_name', 'annotation'])
    features = df.values
    features = features.copy().T
    normalized_features = []
    train_means = []
    train_std = []
    for i, feature in enumerate(features):
        m = None
        std = None
        if means is None:
            m = np.mean(feature)
            std = np.std(feature)
            train_means.append(m)
            train_std.append(std)
        else:
            m = means[i]
            std = stds[i]
        feature -= m
        feature /= std
        normalized_features.append(feature)
    normalized_features = np.array(normalized_features).T
    print(normalized_features.shape)
    X_images = []

    for file in tqdm(file_names):
        img = cv2.imread(img_path + file)
        img = cv2.resize(img, (224, 224), cv2.INTER_CUBIC)
        X_images.append(img.astype('float32')/255)

    return np.array(X_images).astype('float32'), normalized_features, np.array(to_categorical(Y)), train_means, train_std

In [0]:
def calc_accuracy(Y, Y_pred):
    total = len(Y)
    if len(Y) != len(Y_pred):
        print("Shapes for predictions and ground truth don't match")
        return
    correct = sum([int(Y_pred[i] == Y[i]) for i in range(total)])
    return correct * 100 / total
from sklearn.metrics import mean_squared_error, classification_report
def eval(pred, gt):
    pred = np.argmax(pred, axis=1)
    accuracy = calc_accuracy(gt, list(pred))
    mse = mean_squared_error(gt, list(pred))
    print("Classification report for - \n{}:\n{}\n".format(None, metrics.classification_report(gt, list(pred))))
    print("MSE = " + str(mse))
    print("Accuracy = " + str(accuracy))

In [0]:
X_images_train, X_features_train, Y_train, train_means, train_std = read_data_set('./data/ExtraCredit_Train.csv', './data/TrainData/', None, None)

In [0]:
print(X_features_train[:10])

In [0]:
X_images_test, X_features_test, Y_test, _, _ = read_data_set('./data/ExtraCredit_Test.csv', './data/TrainData/', train_means, train_std)

In [0]:
def create_features_channel(X_features):
    channels = []
    dim_x, dim_y = 32, 32
    break_x = int(dim_x/4)
    break_y = int(dim_y/3)
    for x in X_features:
        r = np.zeros((dim_x, dim_y))
        f_ind = 0

        for i in range(4):
            for j in range(3):
                r[j * break_y: (j+1) * break_y, i * break_x: (i+1) * break_x] = x[f_ind]
                f_ind += 1

        g = np.zeros((dim_x, dim_y))
        f_ind = 0

        for i in range(3):
            for j in range(4):
                g[j * break_x: (j+1) * break_x, i * break_y: (i+1) * break_y] = x[f_ind]
                f_ind += 1

        b = np.zeros((dim_x, dim_y))
        f_ind = 0

        for j in range(3, -1, -1):
            for i in range(3):
                b[j * break_x: (j+1) * break_x, i * break_y: (i+1) * break_y] = x[f_ind]
                f_ind += 1

        channel = cv2.merge((r, g, b)).astype('float32')
        channels.append(channel)
    return np.array(channels)

In [0]:
X_channel_features_train = create_features_channel(X_features_train)
X_channel_features_test = create_features_channel(X_features_test)
print(X_channel_features_train.shape)

In [0]:
def model_1():
    # activity_regularizer = None
    activity_regularizer = regularizers.l2(0.00001)
    base_activity_regularizer = regularizers.l1(0.0001)

    features = Input(shape=(12, ))
    # x = Dense(12, activation='relu', activity_regularizer=None)(features)
    # x = GaussianNoise(0.001)(features)
    x = Dense(1000, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(features)
    # x = Dropout(0.1)(x)
    x = Dense(180, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    # x = Dropout(0.1)(x)
    x = Dense(12, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    
    # x = GaussianNoise(0.04)(x)
    # x = Dense(120, activation='relu', activity_regularizer=None)(x)
    # x = Dense(12, activation='relu', activity_regularizer=None)(x)
    # x = Dense(12, activation='relu', activity_regularizer=None)(x)
    # x = Dropout(0.5)(x)
    y = GaussianNoise(0.075)(features)
    # y = Dense(12, activation='relu', activity_regularizer=None, bias_regularizer=None)(y)
    
    x = Dense(5, activation='softmax', bias_regularizer=None)(concatenate([x, y]))
    model = Model(features, x)
    return model



In [0]:
model_weather = model_1()
model_weather.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model_weather.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(cnn_model, to_file='model.png')

In [0]:
history = model_weather.fit(X_features_train, Y_train, validation_data=(X_features_test, Y_test), epochs=200,  verbose=1, class_weight=class_weights, callbacks=callbacks, shuffle=True, batch_size=None)


In [0]:
weather_only_pred = model_weather.predict(X_features_test)
print(list(np.argmax(weather_only_pred, axis=1)))
eval(weather_only_pred, list(np.argmax(Y_test, axis=1)))

In [0]:
plt.figure(dpi=200)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(dpi=200)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_generator = ImageDataGenerator(fill_mode='reflect', rotation_range=15, channel_shift_range=0.2, brightness_range=[0.5, 1.3], horizontal_flip=True, vertical_flip=True, zoom_range=[0.5, 1], dtype='float32', rescale=1./255, samplewise_center=True, samplewise_std_normalization=True)
test_data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True, rescale=1./255)


In [0]:

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5, restore_best_weights=True)
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
callbacks = [es]
class_counts = Counter(list(np.argmax(Y_train, axis=1)))
class_weights = {}
sub_total = len(list(Y_train))/2
for i in range(5):
    class_weights[i] = ((sub_total / class_counts[i]) + 5) / 6
print("Final weights = " + str(class_weights) )

In [0]:
def model_2():
    # activity_regularizer = None
    top_regularizer = regularizers.l2(0.001)
    activity_regularizer = regularizers.l1(0.01)


    base = tf.keras.applications.MobileNet(include_top=False)


    for layer in base.layers:
      for attr in ['activity_regularizer']:
        if hasattr(layer, attr):
          # print(attr)
          setattr(layer, attr, activity_regularizer)
    



    

    x = base.output
    x = GlobalMaxPooling2D()(x)
    # x = GaussianNoise(0.1)(x)
    # x = Dropout(0.8)(x)
    x = Dense(300, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    # x = Dropout(0.3)(x)
    x = Dense(100, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    # x = Dropout(0.2)(x)

    # y = GaussianNoise(0.075)(features)
    # y = Dense(12, activation='relu', activity_regularizer=None, bias_regularizer=None)(y)
    
    x = Dense(5, activation='softmax', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    model = Model(base.input, x)
    return model




In [0]:
cnn_model = model_2()
cnn_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])
cnn_model.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(cnn_model, to_file='model.png')

In [0]:

# train = train_data_generator.flow(X_images_train, Y_train, batch_size=40, shuffle=True)
# test = test_data_generator.flow(X_images_test, Y_test, batch_size=25, shuffle=False)
history = cnn_model.fit(X_images_train, Y_train, validation_data=(X_images_test, Y_test), epochs=200,  verbose=1, class_weight=class_weights, callbacks=callbacks, shuffle=True, batch_size=40, validation_steps=5)


In [0]:

pred_cnn_only = cnn_model.predict(X_images_test)
print(list(np.argmax(pred_cnn_only, axis=1)))
eval(pred_cnn_only, list(np.argmax(Y_test, axis=1)))

In [0]:
plt.figure(dpi=200)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(dpi=200)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [0]:
def ensemble():


    # activity_regularizer = None
    activity_regularizer = regularizers.l2(0.00001)
    base_activity_regularizer = regularizers.l1(0.0001)

    features = Input(shape=(12, ))
    a = Dense(1000, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(features)

    a = Dense(180, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(a)

    a = Dense(12, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(a)

    b = GaussianNoise(0.075)(features)
    # y = Dense(12, activation='relu', activity_regularizer=None, bias_regularizer=None)(y)
    
    out_1 = Dense(5, activation='relu', bias_regularizer=None)(concatenate([a, b]))


    # activity_regularizer = None
    top_regularizer = regularizers.l2(0.003)
    activity_regularizer = regularizers.l2(0.01)


    base = tf.keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3))


    for layer in base.layers:
      for attr in ['activity_regularizer']:
        if hasattr(layer, attr):
          # print(attr)
          setattr(layer, attr, activity_regularizer)
    



    

    x = base.output
    x = GlobalMaxPooling2D()(x)
    # x = GaussianNoise(0.1)(x)
    # x = Dropout(0.1)(x)
    x = Dense(300, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    # x = Dropout(0.3)(x)
    x = Dense(100, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)
    # x = Dropout(0.2)(x)

    # y = GaussianNoise(0.075)(features)
    # y = Dense(12, activation='relu', activity_regularizer=None, bias_regularizer=None)(y)
    
    out_2 = Dense(5, activation='relu', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(x)

    pred = Dense(5, activation='softmax', activity_regularizer=activity_regularizer, bias_regularizer=activity_regularizer)(concatenate([out_1, out_2]))

    model = Model([base.input, features], pred)
    return model

In [0]:
ensmble_model = ensemble()
ensmble_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
# ensmble_model.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(ensmble, to_file='model.png')

In [0]:
history_1 = ensmble_model.fit([X_images_train, X_features_train], Y_train, validation_data=([X_images_test, X_features_test], Y_test), epochs=200,  verbose=1, class_weight=class_weights, callbacks=callbacks, shuffle=True, batch_size=40, validation_steps=5)


In [0]:

pred_combined_only = ensmble_model.predict([X_images_test, X_features_test])

print(list(np.argmax(pred_combined_only, axis=1)))
eval(pred_combined_only, list(np.argmax(Y_test, axis=1)))

In [0]:
plt.figure(dpi=200)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(dpi=200)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()